In [2]:
import numpy as np
from scipy import signal
import soundfile as sf
import sounddevice as sd
import tkinter as tk
from tkinter import ttk, filedialog
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import queue
import threading
import time
import os
from PIL import Image, ImageTk

class AudioFile:
    def __init__(self):
        self.audio_data = None
        self.sample_rate = None
        self.current_position = 0
        self.duration = 0
        self.is_playing = False
        
    def load_file(self, filename):
        """Đọc file audio"""
        try:
            audio_data, sample_rate = sf.read(filename)
            # Chuyển đổi stereo thành mono nếu cần
            if len(audio_data.shape) > 1:
                audio_data = np.mean(audio_data, axis=1)
            
            self.audio_data = audio_data
            self.sample_rate = sample_rate
            self.current_position = 0
            self.duration = len(audio_data) / sample_rate
            return True
        except Exception as e:
            print(f"Error loading audio file: {e}")
            return False
            
    def get_next_chunk(self, chunk_size):
        """Lấy chunk tiếp theo của audio để xử lý"""
        if self.current_position >= len(self.audio_data):
            return None
            
        end_pos = min(self.current_position + chunk_size, len(self.audio_data))
        chunk = self.audio_data[self.current_position:end_pos]
        self.current_position = end_pos
        
        # Pad với 0 nếu chunk cuối không đủ độ dài
        if len(chunk) < chunk_size:
            chunk = np.pad(chunk, (0, chunk_size - len(chunk)))
            
        return chunk
        
    def seek(self, position):
        """Di chuyển đến vị trí cụ thể trong file (tính bằng giây)"""
        self.current_position = int(position * self.sample_rate)
        
class AudioPlayer:
    def __init__(self, sample_rate, frame_size):
        self.sample_rate = sample_rate
        self.frame_size = frame_size
        self.stream = None
        self.audio_file = AudioFile()
        
    def play(self, callback):
        """Bắt đầu phát audio với callback xử lý"""
        if self.stream is None or not self.stream.active:
            self.stream = sd.OutputStream(
                channels=1,
                samplerate=self.sample_rate,
                blocksize=self.frame_size,
                callback=callback
            )
            self.stream.start()
            
    def stop(self):
        """Dừng phát audio"""
        if self.stream is not None and self.stream.active:
            self.stream.stop()
            self.stream.close()
            self.stream = None
            
    def load_file(self, filename):
        """Load file audio mới"""
        return self.audio_file.load_file(filename)

class AudioVisualizer:
    def __init__(self, frame, sample_rate):
        self.sample_rate = sample_rate
        
        # Tạo figure cho matplotlib
        self.fig = Figure(figsize=(10, 6))
        
        # Subplot cho dạng sóng
        self.ax_wave = self.fig.add_subplot(211)
        self.ax_wave.set_title('Waveform')
        self.ax_wave.set_ylim(-1, 1)
        self.ax_wave.set_xlim(0, 1024)
        self.wave_line, = self.ax_wave.plot([], [], 'b-', lw=1)
        
        # Subplot cho phổ tần số
        self.ax_spectrum = self.fig.add_subplot(212)
        self.ax_spectrum.set_title('Frequency Spectrum')
        self.ax_spectrum.set_ylim(-60, 20)
        self.ax_spectrum.set_xlim(20, 20000)
        self.ax_spectrum.set_xscale('log')
        self.spectrum_line, = self.ax_spectrum.plot([], [], 'g-', lw=1)
        
        # Thêm canvas vào frame
        self.canvas = FigureCanvasTkAgg(self.fig, master=frame)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        
        self.fig.tight_layout()
        
    def update(self, audio_data):
        # Cập nhật dạng sóng
        self.wave_line.set_data(np.arange(len(audio_data)), audio_data)
        
        # Tính và cập nhật phổ tần số
        spectrum = np.fft.rfft(audio_data)
        freq = np.fft.rfftfreq(len(audio_data), 1/self.sample_rate)
        spectrum_db = 20 * np.log10(np.abs(spectrum) + 1e-10)
        self.spectrum_line.set_data(freq, spectrum_db)
        
        # Vẽ lại canvas
        self.canvas.draw()

class AudioEqualizer:
    def __init__(self):
        # Thiết lập các thông số cơ bản
        self.sample_rate = 44100  # Hz
        self.num_bands = 8
        self.frame_size = 1024
        self.audio_queue = queue.Queue(maxsize=10)
        
        # Tạo các băng tần trung tâm (Hz)
        self.bands = [60, 170, 310, 600, 1000, 3000, 6000, 12000]
        self.gains = np.ones(self.num_bands)
        
        # Khởi tạo bộ lọc FIR cho mỗi băng
        self.filters = self._design_filters()
        
        # Khởi tạo audio player
        self.player = AudioPlayer(self.sample_rate, self.frame_size)
        
    def _design_filters(self):
        filters = []
        nyquist = self.sample_rate / 2
        
        for i, freq in enumerate(self.bands):
            if i == 0:
                high = freq * 1.5
                b = signal.firwin(101, high, fs=self.sample_rate)
            elif i == len(self.bands) - 1:
                low = freq / 1.5
                b = signal.firwin(101, low, fs=self.sample_rate, pass_zero=False)
            else:
                low = freq / 1.5
                high = freq * 1.5
                b = signal.firwin(101, [low, high], fs=self.sample_rate, pass_zero=False)
            
            filters.append(b)
        return filters
    
    def process_audio(self, audio_data):
        """Xử lý tín hiệu audio với các bộ lọc và độ lợi"""
        output = np.zeros_like(audio_data)
        
        for i in range(self.num_bands):
            filtered = signal.lfilter(self.filters[i], [1.0], audio_data)
            output += filtered * self.gains[i]
            
        return output
        
    def audio_callback(self, outdata, frames, time, status):
        """Callback function cho stream audio"""
        if status:
            print(status)
            
        # Lấy chunk tiếp theo từ file
        chunk = self.player.audio_file.get_next_chunk(frames)
        
        if chunk is None:
            # Hết file, dừng phát
            self.player.stop()
            self.player.audio_file.current_position = 0
            raise sd.CallbackStop()
            
        # Xử lý audio và đưa vào output
        processed = self.process_audio(chunk)
        outdata[:] = processed.reshape(-1, 1)
        
        try:
            self.audio_queue.put_nowait(processed)
        except queue.Full:
            pass
            
    def set_gain(self, band_idx, gain_db):
        """Cập nhật độ lợi cho băng tần (dB)"""
        self.gains[band_idx] = 10 ** (gain_db / 20)

class EqualizerGUI:
    def __init__(self, equalizer):
        self.equalizer = equalizer
        self.root = tk.Tk()
        self.root.title("Audio Equalizer")
        
        # Frame chính
        main_frame = ttk.Frame(self.root)
        main_frame.pack(expand=True, fill=tk.BOTH, padx=10, pady=10)
        
        # Frame cho file control
        file_frame = ttk.Frame(main_frame)
        file_frame.pack(side=tk.TOP, fill=tk.X, pady=5)
        
        # Nút chọn file
        ttk.Button(file_frame, text="Open File", command=self.open_file).pack(side=tk.LEFT, padx=5)
        
        # Labels hiển thị thông tin file
        self.file_label = ttk.Label(file_frame, text="No file loaded")
        self.file_label.pack(side=tk.LEFT, padx=5)
        
        # Progress bar
        self.progress = ttk.Scale(file_frame, from_=0, to=100, orient=tk.HORIZONTAL)
        self.progress.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=5)
        self.progress.bind("<Button-1>", self.seek_file)
        
        # Frame cho visualizer
        viz_frame = ttk.Frame(main_frame)
        viz_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True)
        
        # Khởi tạo visualizer
        self.visualizer = AudioVisualizer(viz_frame, equalizer.sample_rate)
        
        # Frame cho các điều khiển
        control_frame = ttk.Frame(main_frame)
        control_frame.pack(side=tk.BOTTOM, fill=tk.X, pady=10)
        
        # Frame cho các slider
        slider_frame = ttk.Frame(control_frame)
        slider_frame.pack(side=tk.TOP, fill=tk.X)
        
        # Tạo các slider điều khiển
        self.sliders = []
        for i, freq in enumerate(equalizer.bands):
            frame = ttk.Frame(slider_frame)
            frame.pack(side=tk.LEFT, padx=5)
            
            slider = ttk.Scale(frame, from_=12, to=-12, length=200,
                             orient=tk.VERTICAL, command=lambda x, i=i: self.update_gain(i, x))
            slider.set(0)
            slider.pack()
            
            label = ttk.Label(frame, text=f"{freq}Hz")
            label.pack()
            
            self.sliders.append(slider)
        
        # Nút điều khiển
        button_frame = ttk.Frame(control_frame)
        button_frame.pack(side=tk.BOTTOM, pady=10)
        
        # Thêm icons cho các nút
        self.play_icon = "▶"
        self.pause_icon = "⏸"
        self.stop_icon = "⏹"
        
        self.play_button = ttk.Button(button_frame, text=self.play_icon, 
                                    command=self.toggle_play, width=3)
        self.play_button.pack(side=tk.LEFT, padx=5)
        
        ttk.Button(button_frame, text=self.stop_icon, 
                  command=self.stop_audio, width=3).pack(side=tk.LEFT, padx=5)
        
        # Khởi động thread cập nhật visualization
        self.running = True
        self.update_thread = threading.Thread(target=self.update_visualization)
        self.update_thread.daemon = True
        self.update_thread.start()
        
        # Timer để cập nhật progress bar
        self.root.after(100, self.update_progress)
        
    def open_file(self):
        """Mở file dialog để chọn file audio"""
        filetypes = [
            ("Audio files", "*.wav;*.mp3;*.ogg"),
            ("WAV files", "*.wav"),
            ("MP3 files", "*.mp3"),
            ("OGG files", "*.ogg"),
            ("All files", "*.*")
        ]
        
        filename = filedialog.askopenfilename(filetypes=filetypes)
        if filename:
            if self.equalizer.player.load_file(filename):
                self.file_label.config(text=os.path.basename(filename))
                self.progress.config(to=self.equalizer.player.audio_file.duration)
                self.play_audio()
                
    def toggle_play(self):
        """Toggle giữa play và pause"""
        if self.equalizer.player.stream is None or not self.equalizer.player.stream.active:
            self.play_audio()
            self.play_button.config(text=self.pause_icon)
        else:
            self.pause_audio()
            self.play_button.config(text=self.play_icon)
            
    def play_audio(self):
        """Bắt đầu phát audio"""
        if self.equalizer.player.audio_file.audio_data is not None:
            self.equalizer.player.play(self.equalizer.audio_callback)
            
    def pause_audio(self):
        """Tạm dừng phát audio"""
        self.equalizer.player.stop()
        
    def stop_audio(self):
        """Dừng và reset audio"""
        self.equalizer.player.stop()
        self.equalizer.player.audio_file.current_position = 0
        self.play_button.config(text=self.play_icon)
        
    def seek_file(self, event):
        """Xử lý seek trong file audio"""
        if self.equalizer.player.audio_file.audio_data is not None:
            position = self.progress.get()
            self.equalizer.player.audio_file.seek(position)
            
    def update_progress(self):
        """Cập nhật progress bar"""
        if self.equalizer.player.stream is not None and self.equalizer.player.stream.active:
            current_time = self.equalizer.player.audio_file.current_position / self.equalizer.sample_rate
            self.progress.set(current_time)
        self.root.after(100, self.update_progress)
        
    def update_visualization(self):
        """Thread cập nhật visualization"""
        while self.running:
            try:
                audio_data = self.equalizer.audio_queue.get_nowait()
                self.visualizer.update(audio_data)
            except queue.Empty:
                time.sleep(0.01)
            except Exception as e:
                print(f"Visualization error: {e}")
                
    def update_gain(self, band_idx, value):
        self.equalizer.set_gain(band_idx, float(value))
        
    def run(self):
        try:
            self.root.mainloop()
        finally:
            self.running = False
            self.equalizer.player.stop()

if __name__ == "__main__":
    eq = AudioEqualizer()
    gui = EqualizerGUI(eq)
    gui.run()